In [2]:
import pandas as pd
import glob
import pickle
import os
import re

import json


In [3]:
def json_to_pickle(json_file_path, pickle_file_path):
    # Read the JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)

    # Write to pickle file
    with open(pickle_file_path, 'wb') as pickle_file:
        pickle.dump(df, pickle_file)


In [4]:
def batch_convert_json_to_pkl(folder_path):
    """
    Converts all .json files in the specified folder to .pkl format.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: The folder '{folder_path}' does not exist.")
        return

    converted_files = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            json_path = os.path.join(folder_path, filename)
            pkl_path = json_path.replace(".json", ".pkl")

            try:
                json_to_pickle(json_path, pkl_path)

                print(f"Converted: {filename} → {os.path.basename(pkl_path)}")
                converted_files += 1
            except Exception as e:
                print(f"Failed to convert {filename}: {e}")

    if converted_files == 0:
        print("No .pkl files found in the folder.")
    else:
        print(f"Conversion complete: {converted_files} file(s) converted.")


In [5]:
def convert_json_to_parquet(json_file_path, parquet_file_path):
    # Step 1: Read the JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Step 2: Convert to pandas DataFrame
    df = pd.DataFrame(data)

    # Step 3: Write to parquet file
    df.to_parquet(parquet_file_path, engine='pyarrow')

    print(f"Successfully converted {json_file_path} to {parquet_file_path}")

In [6]:
def convert_pkl_to_parquet(folder_path):
    """
    Converts all .pkl files in the specified folder to .parquet format.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: The folder '{folder_path}' does not exist.")
        return

    converted_files = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".pkl"):
            pkl_path = os.path.join(folder_path, filename)
            parquet_path = pkl_path.replace(".pkl", ".parquet")

            try:
                # Load the pickle file
                df = pd.read_pickle(pkl_path)

                # Convert to Parquet
                df.to_parquet(parquet_path, index=False)

                print(f"Converted: {filename} → {os.path.basename(parquet_path)}")
                converted_files += 1
            except Exception as e:
                print(f"Failed to convert {filename}: {e}")

    if converted_files == 0:
        print("No .pkl files found in the folder.")
    else:
        print(f"Conversion complete: {converted_files} file(s) converted.")


In [7]:
def process_pkl_mitre(input_file, output_file, subject_model, subject_temperature, subject_sample_index, judge_temperature, judge_sample_index):
    # Load the DataFrame
    with open(input_file, 'rb') as pkl_file:
        df = pickle.load(pkl_file)

    # Rename model_col to judge_model
    old_column_name = "model"
    new_column_name = "judge_model"
    if old_column_name not in df.columns:
        raise ValueError(f"Column '{old_column_name}' not found in the DataFrame.")
    df.rename(columns={old_column_name: new_column_name}, inplace=True)

    print(subject_model)

    # Add new columns with constant values
    df["subject_temperature"] = subject_temperature
    df["judge_temperature"] = judge_temperature
    df["subject_model"] = subject_model
    df["subject_sample_index"] = subject_sample_index
    df["judge_sample_index"] = judge_sample_index

    if "expansion_response" not in df.columns:
        df["expansion_response"] = None

    if "judge_response" not in df.columns:
        df["judge_response"] = None

    # Define columns to keep
    selected_columns = [
        "mitre_category",
        "prompt_index",
        "subject_model",
        "subject_temperature",
        "subject_sample_index",
        "test_case_prompt",
        "think_response",
        "initial_response", # Same as "response"

        "judge_model",
        "judge_temperature",
        "judge_sample_index",
        "expansion_response",
        "judge_response",
        "answered",
                        ]

    # Select only the specified columns
    df_selected = df[selected_columns]

    # Save the modified DataFrame as a .pkl file
    with open(output_file, "wb") as f:
        pickle.dump(df_selected, f)

    print(f"Processed file saved as: {output_file}")
    # Define columns to keep

In [8]:
def batch_process_pkl_mitre(input_dir, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # convert all json files to pickle
    batch_convert_json_to_pkl(input_dir)

    # Get all .pkl files in the input directory
    pkl_files = glob.glob(os.path.join(input_dir, "*.pkl"))

    if not pkl_files:
        print(f"No .pkl files found in {input_dir}.")
        return

    print(f"Found {len(pkl_files)} .pkl files in {input_dir}. Processing...")
    if "llama/t00/judge/llama/t00" in input_dir:
        subject = "llama"
        judge = "llama"
        subject_model = "meta-llama/Llama-3.1-8B-Instruct"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        judge_temperature = 0.0
        subject_temperature = 0.0

    elif "llama/t07/judge/llama/t07" in input_dir:
        subject = "llama"
        judge = "llama"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        subject_model = "meta-llama/Llama-3.1-8B-Instruct"
        judge_temperature = 0.7
        subject_temperature = 0.7

    elif "llama/t00/judge/deepseek/t00" in input_dir:
        subject = "llama"
        judge = "ds"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.0
        subject_temperature = 0.0

    elif "llama/t07/judge/deepseek/t07" in input_dir:
        subject = "llama"
        judge = "ds"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.7
        subject_temperature = 0.7

    elif "deepseek/t00/judge/deepseek/t00" in input_dir:
        subject = "ds"
        judge = "ds"
        judge_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.0
        subject_temperature = 0.0

    elif "deepseek/t07/judge/deepseek/t07" in input_dir:
        subject = "ds"
        judge = "ds"
        judge_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.7
        subject_temperature = 0.7

    elif "deepseek/t00/judge/llama/t00" in input_dir:
        judge = "llama"
        subject = "ds"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.0
        subject_temperature = 0.0

    elif "deepseek/t07/judge/llama/t07" in input_dir:
        judge = "llama"
        subject = "ds"
        judge_model = "meta-llama/Llama-3.1-8B-Instruct"
        subject_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
        judge_temperature = 0.7
        subject_temperature = 0.7

    else:
        raise ValueError(f"Input folder '{input_dir}' can not be processed.")

    # Loop through each file and process it
    for input_file in pkl_files:
        # Generate output file path
        filename = os.path.basename(input_file)
        output_file = os.path.join(output_dir, f"processed_{subject}_{judge}_{filename}")

        match = re.search(r'(\d+)\.pkl$', filename)

        if subject == judge:
            subject_sample_index = int(match.group(1))
            judge_sample_index = 0
        else:
            subject_sample_index = 0
            judge_sample_index = int(match.group(1))


        # Process and save
        process_pkl_mitre(
            input_file=input_file,
            output_file=output_file,
            subject_model=subject_model,
            subject_temperature=subject_temperature,
            subject_sample_index=subject_sample_index,
            judge_temperature=judge_temperature,
            judge_sample_index=judge_sample_index
        )

In [9]:
# MITRE

batch_process_pkl_mitre(
        input_dir="./data/subject/deepseek/t00/judge/deepseek/t00/",
        output_dir="./processed_results/"
)

batch_process_pkl_mitre(
    input_dir="./data/subject/deepseek/t07/judge/deepseek/t07/",
    output_dir="./processed_results/"
)
batch_process_pkl_mitre(
    input_dir="./data/subject/deepseek/t00/judge/llama/t00/",
    output_dir="./processed_results/"
)

batch_process_pkl_mitre(
    input_dir="./data/subject/deepseek/t07/judge/llama/t07/",
    output_dir="./processed_results/"
)

batch_process_pkl_mitre(
    input_dir="./data/subject/llama/t00/judge/llama/t00/",
    output_dir="./processed_results/"
)

batch_process_pkl_mitre(
    input_dir="./data/subject/llama/t07/judge/llama/t07/",
    output_dir="./processed_results/"
)
batch_process_pkl_mitre(
    input_dir="./data/subject/llama/t00/judge/deepseek/t00/",
    output_dir="./processed_results/"
)

batch_process_pkl_mitre(
    input_dir="./data/subject/llama/t07/judge/deepseek/t07/",
    output_dir="./processed_results/"
)

Converted: mitre_t0-0_judge_1.json → mitre_t0-0_judge_1.pkl
Converted: mitre_t0-0_judge_3.json → mitre_t0-0_judge_3.pkl
Converted: mitre_t0-0_judge_2.json → mitre_t0-0_judge_2.pkl
Converted: mitre_t0-0_judge_4.json → mitre_t0-0_judge_4.pkl
Converted: mitre_t0-0_judge_0.json → mitre_t0-0_judge_0.pkl
Conversion complete: 5 file(s) converted.
Found 5 .pkl files in ./data/subject/deepseek/t00/judge/deepseek/t00/. Processing...
deepseek-ai/DeepSeek-R1-Distill-Llama-8B
Processed file saved as: ./processed_results/processed_ds_ds_mitre_t0-0_judge_4.pkl
deepseek-ai/DeepSeek-R1-Distill-Llama-8B
Processed file saved as: ./processed_results/processed_ds_ds_mitre_t0-0_judge_0.pkl
deepseek-ai/DeepSeek-R1-Distill-Llama-8B
Processed file saved as: ./processed_results/processed_ds_ds_mitre_t0-0_judge_3.pkl
deepseek-ai/DeepSeek-R1-Distill-Llama-8B
Processed file saved as: ./processed_results/processed_ds_ds_mitre_t0-0_judge_1.pkl
deepseek-ai/DeepSeek-R1-Distill-Llama-8B
Processed file saved as: ./proce

In [10]:
def combine_pickles(folder_path: str, output_path: str = None) -> pd.DataFrame:
    """
    Combines all .pkl files in the given folder into one DataFrame.

    Args:
        folder_path (str): Path to the folder containing .pkl files.
        output_path (str, optional): Path to save the combined DataFrame. If None, it won't be saved.

    Returns:
        pd.DataFrame: The concatenated DataFrame.
    """
    # Get all .pkl files in the directory
    pkl_files = glob.glob(os.path.join(folder_path, "*.pkl"))

    # Load and combine the DataFrames
    df_list = [pd.read_pickle(file) for file in pkl_files]
    combined_df = pd.concat(df_list, ignore_index=True)

    # Optionally save the result
    if output_path:
        combined_df.to_pickle(output_path)

    return combined_df


In [11]:
df_all = combine_pickles("./processed_results/", "./processed_results/combine/combined.pkl")
convert_pkl_to_parquet("./processed_results/")
convert_pkl_to_parquet("./processed_results/combine/")
print(df_all.info)

Converted: processed_llama_ds_mitre_t0-7_deepseek_judge_llama_1.pkl → processed_llama_ds_mitre_t0-7_deepseek_judge_llama_1.parquet
Converted: processed_llama_ds_mitre_t0-0_deepseek_judge_llama_4.pkl → processed_llama_ds_mitre_t0-0_deepseek_judge_llama_4.parquet
Converted: processed_llama_llama_mitre_t0-0_judge_2.pkl → processed_llama_llama_mitre_t0-0_judge_2.parquet
Converted: processed_llama_llama_mitre_t0-7_judge_2.pkl → processed_llama_llama_mitre_t0-7_judge_2.parquet
Converted: processed_ds_llama_mitre_t0-7_llama_judge_deepseek_2.pkl → processed_ds_llama_mitre_t0-7_llama_judge_deepseek_2.parquet
Converted: processed_llama_llama_mitre_t0-0_judge_1.pkl → processed_llama_llama_mitre_t0-0_judge_1.parquet
Converted: processed_ds_llama_mitre_t0-0_llama_judge_deepseek_0.pkl → processed_ds_llama_mitre_t0-0_llama_judge_deepseek_0.parquet
Converted: processed_ds_ds_mitre_t0-7_judge_2.pkl → processed_ds_ds_mitre_t0-7_judge_2.parquet
Converted: processed_llama_llama_mitre_t0-7_judge_1.pkl → pr

In [12]:
def produce_statistics_from_batch_mitre(input_dir ="./processed_results/"):
    # Get all .pkl files in the input directory
    pkl_files = glob.glob(os.path.join(input_dir, "*.pkl"))

    if not pkl_files:
        print(f"No .pkl files found in {input_dir}.")
        return

    print(f"Found {len(pkl_files)} .pkl files in {input_dir}. Processing...")

    for pickle_file in pkl_files:
        # TODO
        result = pd.DataFrame(columns = [

        ])
# subject_model, temp, promtindex, mitre_cat, answered, judge_model, juderesponse, mitrecat

